In [7]:
import pandas as pd

In [9]:
df_meta = pd.read_csv("dash_dataset.csv")[["meta_id", "sex", "age", "BMI", "study", "length_of_fasting", "length_of_fasting_cat", "age_cat", "BMI_cat"]].drop_duplicates()
vars_to_drop = ["ketonuria [mg/dL]", "physical activity [min/day]", "sleep quality [0-10]", "PWB [0-10]", "PSA total [ng/mL]", "INR [-]"]

df_long = (
    pd.read_csv("dash_dataset.csv")
    .drop(columns=["length_of_fasting_cat", "sex", "age_cat", 
                   "BMI_cat", "age", "length_of_fasting", "BMI", "study"])
    .melt(
        id_vars = ["meta_id"]
    )
    .assign(    
            # Détecter "Pre", "Pobst" ou le signe delta (Δ ou 'Δ') même sans unités dans le nom de variable
            timepoint = lambda x: (
                x.variable.str.extract(r"(Pre|Post|Δ|\u0394)", expand=False)
            ),
            variable = lambda x: (
                x.variable
                 .str.replace(r" Pre$| Post$| Δ$| Δ$", "", regex=True)
            ),
    )
    .query("variable not in @vars_to_drop")

    .merge(df_meta, on="meta_id", how="left")
)

In [13]:
import sys
import numpy as np

pivot_df = (
    df_long
     .pivot_table(
        index = ["meta_id", "sex", "age_cat", "BMI_cat", "length_of_fasting_cat", "timepoint"], 
        columns = [ "variable"], 
        values = "value"
    )
     .reset_index()
).to_csv("dash_dataset_wide.csv")

# memory_usage_bytes = pivot_df.memory_usage(deep=True).sum()
# memory_usage_mb = memory_usage_bytes / (1024 ** 2)

# print(f"Taille du DataFrame pivoté : {memory_usage_mb:.2f} Mo")

In [16]:
vars_to_drop = ["ketonuria [mg/dL]", "physical activity [min/day]", "sleep quality [0-10]", "PWB [0-10]", "PSA total [ng/mL]", "INR [-].query"]
(pd.read_csv("dash_dataset.csv")
    .drop(columns=["length_of_fasting_cat", "sex", "age_cat", 
                   "BMI_cat", "age", "length_of_fasting", "BMI", "study"])
    .melt(
        id_vars = ["meta_id"]
    )
    .assign(    
            # Détecter "Pre", "Pobst" ou le signe delta (Δ ou 'Δ') même sans unités dans le nom de variable
            timepoint = lambda x: (
                x.variable.str.extract(r"(Pre|Post|Δ|\u0394)", expand=False)
            ),
            variable = lambda x: (
                x.variable
                 .str.replace(r" Pre$| Post$| Δ$| Δ$", "", regex=True)
            ),
    )
    .query("variable not in @vars_to_drop")
    
    .merge(df_meta, on="meta_id", how="left")
).to_csv("df_long.csv")

In [ ]:
import pandas as pd
df_long = pd.read_csv("df_long.csv", index_col=0, parse_dates=False)

In [2]:
df_long#.to_csv("df_long.csv")

NameError: name 'df_long' is not defined

In [ ]:
px.scatter(df_long, x="length_of", y="BMI", color="sex")

array(['ALP [U/L]', 'BMI [kg/m²]', 'CRP hs [mg/L]', 'Ca [mg/dL]',
       'DBP [mmHg]', 'ESR 1H [mm/h]', 'ESR 2H [mm/h]', 'EWB [0-10]',
       'FLI', 'GFR [mL/min/1.73m²]', 'GGT [U/L]', 'GOT AST [U/L]',
       'GPT ALT [U/L]', 'HBA1C [%]', 'HBA1C [mmol/mol]', 'HDL [mg/dL]',
       'HR [/min]', 'INR [-]', 'K [mmol/L]', 'LDL HDL ratio [-]',
       'LDL [mg/dL]', 'MCH [pg]', 'MCHC [g/dL]', 'MCV [fL]', 'Mg [mg/dL]',
       'Na [mmol/L]', 'PSA total [ng/mL]', 'PTT [s]', 'PWB [0-10]',
       'SBP [mmHg]', 'TC [mg/dL]', 'TG [mg/dL]', 'TSH [µU/mL]', 'WC [cm]',
       'creatinine [mg/dL]', 'erythrocytes [T/L]', 'glucose [mg/dL]',
       'hematocrit [%]', 'hemoglobin [g/dL]', 'hunger [0-10]',
       'ketonuria [mg/dL]', 'leukocytes [G/L]',
       'physical activity [min/day]', 'quick [%]', 'sleep quality [0-10]',
       'thrombocytes [G/L]', 'urea [mg/dL]', 'uric acid [mg/dL]',
       'vitamin B12 [pg/mL]', 'vitamin D [ng/mL]', 'weight [kg]'],
      dtype=object)

In [35]:
BMI = 27
age =55
sex = "M"
parameter = 'ALP [U/L]'
Pre_value = 70
df_pat =(
    df_long
    .query("BMI.between(@BMI - 2, @BMI + 2) and age.between(@age - 7, @age + 7) and sex == @sex")

    
)
df_pat_param = df_pat.query("variable == @parameter")
df_pat

,meta_id,length_of_fasting_cat,sex,age_cat,BMI_cat,age,length_of_fasting,BMI,study,variable,value,timepoint
13,101-marbella,3-7 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),53.28,7.0,25.465372,marbella,ALP [U/L],NaN,Pre
52,1042-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),52.69,13.0,26.612713,uberlingen,ALP [U/L],72.0,Pre
228,1203-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),54.20,9.0,28.468628,uberlingen,ALP [U/L],51.0,Pre
247,1222-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),55.65,11.0,28.395062,uberlingen,ALP [U/L],74.0,Pre
253,1228-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),59.17,11.0,26.948463,uberlingen,ALP [U/L],54.0,Pre
...,...,...,...,...,...,...,...,...,...,...,...,...
374704,861-uberlingen,3-7 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),53.76,6.0,26.153990,uberlingen,weight [kg],-3.1,Δ
374711,868-uberlingen,3-7 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),49.08,8.0,27.598650,uberlingen,weight [kg],-4.8,Δ
374808,96-marbella,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),57.91,10.0,27.382125,marbella,weight [kg],-6.0,Δ
374836,987-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),56.64,11.0,26.258270,uberlingen,weight [kg],-4.7,Δ


In [45]:
df_pat.variable.unique()

array(['ALP [U/L]', 'BMI [kg/m²]', 'CRP hs [mg/L]', 'Ca [mg/dL]',
       'DBP [mmHg]', 'ESR 1H [mm/h]', 'ESR 2H [mm/h]', 'EWB [0-10]',
       'FLI', 'GFR [mL/min/1.73m²]', 'GGT [U/L]', 'GOT AST [U/L]',
       'GPT ALT [U/L]', 'HBA1C [%]', 'HBA1C [mmol/mol]', 'HDL [mg/dL]',
       'HR [/min]', 'INR [-]', 'K [mmol/L]', 'LDL HDL ratio [-]',
       'LDL [mg/dL]', 'MCH [pg]', 'MCHC [g/dL]', 'MCV [fL]', 'Mg [mg/dL]',
       'Na [mmol/L]', 'PSA total [ng/mL]', 'PTT [s]', 'PWB [0-10]',
       'SBP [mmHg]', 'TC [mg/dL]', 'TG [mg/dL]', 'TSH [µU/mL]', 'WC [cm]',
       'creatinine [mg/dL]', 'erythrocytes [T/L]', 'glucose [mg/dL]',
       'hematocrit [%]', 'hemoglobin [g/dL]', 'hunger [0-10]',
       'ketonuria [mg/dL]', 'leukocytes [G/L]',
       'physical activity [min/day]', 'quick [%]', 'sleep quality [0-10]',
       'thrombocytes [G/L]', 'urea [mg/dL]', 'uric acid [mg/dL]',
       'vitamin B12 [pg/mL]', 'vitamin D [ng/mL]', 'weight [kg]'],
      dtype=object)

In [78]:
df_pat.query("variable == 'BMI [kg/m²]'")

,meta_id,length_of_fasting_cat,sex,age_cat,BMI_cat,age,length_of_fasting,BMI,study,variable,value,timepoint
7363,101-marbella,3-7 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),53.28,7.0,25.465372,marbella,BMI [kg/m²],25.465372,Pre
7402,1042-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),52.69,13.0,26.612713,uberlingen,BMI [kg/m²],26.612713,Pre
7578,1203-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),54.20,9.0,28.468628,uberlingen,BMI [kg/m²],28.468628,Pre
7597,1222-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),55.65,11.0,28.395062,uberlingen,BMI [kg/m²],28.395062,Pre
7603,1228-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),59.17,11.0,26.948463,uberlingen,BMI [kg/m²],26.948463,Pre
...,...,...,...,...,...,...,...,...,...,...,...,...
14554,861-uberlingen,3-7 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),53.76,6.0,26.153990,uberlingen,BMI [kg/m²],-0.967510,Δ
14561,868-uberlingen,3-7 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),49.08,8.0,27.598650,uberlingen,BMI [kg/m²],-1.603796,Δ
14658,96-marbella,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),57.91,10.0,27.382125,marbella,BMI [kg/m²],-1.791633,Δ
14686,987-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),56.64,11.0,26.258270,uberlingen,BMI [kg/m²],-1.388233,Δ


In [72]:
import statsmodels.formula.api as smf   

variables = ["ALP [U/L]", "LDL [mg/dL]",
              "TG [mg/dL]", "CRP hs [mg/L]",
             "TC [mg/dL]", "SBP [mmHg]",
             "DBP [mmHg]", "glucose [mg/dL]", "HDL [mg/dL]", 
             "GOT AST [U/L]", "GPT ALT [U/L]", "LDL [mg/dL]", "weight [kg]"]
tables = []
for var in variables:
    df_reg = (df_long
            .query("variable == @var").dropna()
            .pivot_table(
                index=["meta_id", "sex", 
                        "BMI", "age"], 
                columns="timepoint", values="value")
            
            ).reset_index()
    formula = f"Post ~ Pre + age + BMI + sex " if var != "weight [kg]" else "Post ~ Pre + age + sex "
    model = smf.ols('Post ~ Pre + age + BMI + sex ', data=df_reg).fit()
    table_res = model.summary2().tables[1].assign(variable=var).reset_index().set_index("variable").reset_index()
    tables.append(table_res)

In [73]:
df_stats =pd.concat(tables)

In [75]:
df_stats.to_csv("df_stats.csv")

In [59]:
table_res#.loc["P>|t|"]

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,4.216189,1.574446,2.677887,0.007517,1.127008,7.305369
sex[T.M],-0.161901,0.508362,-0.318477,0.750182,-1.159345,0.835542
Pre,0.765966,0.013789,55.547345,0.000000,0.738910,0.793022
age,0.065570,0.018570,3.530884,0.000431,0.029133,0.102006
BMI,0.170675,0.043740,3.902046,0.000101,0.084854,0.256495
